In [1]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import random
import os
import sys
import nltk
from collections import Counter
import re
import numpy as np
from argparse import Namespace

In [86]:
#nltk.download('punkt')

In [11]:
flags = Namespace(
    seq_size=32,
    batch_size=16,
    embedding_size=64,
    lstm_size=64,
    gradients_norm=5,
    predict_top_k=5,
)

In [3]:
## rap data
directory = "rap/Jul"
txt = ""
for album in os.listdir(directory):
    dir_album = "{}/{}".format(directory, album)
    for son in os.listdir(dir_album):
        adr = "{}/{}".format(dir_album,son)
        with open(adr, 'rb') as f:
            data = f.read()
            decoded_data = data.decode('utf8')
            txt+=decoded_data



In [12]:
##american dad data
directory = "scripts_american_dad"
txt = ""
for saison in os.listdir(directory):
    dir_saison = "{}/{}".format(directory, saison)
    for ep in os.listdir(dir_saison):
        adr = "{}/{}".format(dir_saison,ep)
        with open(adr, 'r') as f:
            data = f.read()
            txt+=data
            

In [4]:
token = nltk.word_tokenize(txt.lower())
words = Counter(token)
words = sorted(words, key=words.get, reverse=True)
vocab_size = len(words)
word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}
print("vocab size : ",vocab_size)

vocab size :  14562


In [13]:
int_text = [word2idx[w] for w in token]
num_batches = int(len(int_text) / (flags.seq_size * flags.batch_size))
in_text = int_text[:num_batches * flags.batch_size * flags.seq_size]
out_text = np.zeros_like(in_text)
out_text[:-1] = in_text[1:]
out_text[-1] = in_text[0]
in_text = np.reshape(in_text, (flags.batch_size, -1))
out_text = np.reshape(out_text, (flags.batch_size, -1))

In [6]:
def get_batches(in_text, out_text, batch_size, seq_size):
    num_batches = np.prod(in_text.shape) // (seq_size * batch_size)
    for i in range(0, num_batches * seq_size, seq_size):
        yield in_text[:, i:i+seq_size], out_text[:, i:i+seq_size]

In [7]:
class Model(nn.Module):
    def __init__(self, nb_cells, hidden_size, vocab_size, embeddings_dim): 
        super(Model, self).__init__()
        self.gru = nn.GRU(embeddings_dim, hidden_size, nb_cells, batch_first = True)
        self.embeddings = nn.Embedding(vocab_size, embeddings_dim)
        self.hidden_size = hidden_size
        self.nb_cells = nb_cells
        self.dense1 = nn.Linear(hidden_size, vocab_size)

        
    def forward(self, x, hidden):
        embeds = self.embeddings(x)
        gru_out, hidden = self.gru(embeds, hidden)
        out = self.dense1(gru_out)
        return out, hidden
    
        
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.nb_cells, batch_size, self.hidden_size).zero_()
        return hidden

In [8]:
def train(model, max_epochs=5):
    optim = th.optim.Adam(model.parameters(), lr=lr) #Adam adapté aux pb de NLP
    loss_fn = nn.CrossEntropyLoss()
    
    print("Modèle : \n", model)
    for epoch in range(max_epochs):
        batches = get_batches(in_text, out_text, flags.batch_size, flags.seq_size)
        h = model.init_hidden(flags.batch_size)
        i=0
        print("EPOCH {}".format(epoch))
        for x, y in batches:
            i+=1
            optim.zero_grad()
            h = h.data
            x = th.LongTensor(x)
            y = th.LongTensor(y)
            pred, h = model.forward(x, h)
            loss = loss_fn(pred.transpose(1, 2), y)
            h = h.detach()
            loss.backward()  
#             nn.utils.clip_grad_norm_(model.parameters(), flags.gradients_norm)
            optim.step()

            if(i%1000==0):
                print("{}/{}".format(i, num_batches))
            if(i%5000==0):
                print("{}/{}".format(i, num_batches))
                model.eval()
                predict(model, token[:100], word2idx, idx2word, 50)
                print('\n')
                model.train()
                
        print("predict epoch :\n")
        model.eval()
        predict(model, token[:100], word2idx, idx2word, 50)
        model.train()
        print('\n')

In [15]:
###Hyper paramètres
vocab_size = len(word2idx)
embedding_dim = 64
hidden_dim = 64
nb_cells = 1
model = Model(nb_cells,hidden_dim, vocab_size, embedding_dim)
# model.load_state_dict(th.load("model.pth"))
# model2 = Model(nb_cells,hidden_dim, vocab_size, embedding_dim)

# model3 = Model(nb_cells,hidden_dim, vocab_size, embedding_dim)

lr=0.01

In [ ]:
train(model, max_epochs=100)

Modèle : 
 Model(
  (gru): GRU(64, 64, batch_first=True)
  (embeddings): Embedding(14562, 64)
  (dense1): Linear(in_features=64, out_features=14562, bias=True)
)
EPOCH 0
predict epoch :

[ couplet 1 ] et je zone , comme un loup j'ai trop faim , ouais de sous et je serre , comme un fou j'suis déter ' , levez-vous j'ai les cernes , je vois flou pour mes frères , j'me mets saoul franchement , je n'sais plus quoi faire trop d'jaloux j'peux plus les compter faire le mac , à quoi ça sert ? tu peux t'faire fumer ou planter j'charme les miss comme calimero en deux-deux j'prends cent numéros ça sert à rien de faire le héros toujours à la base comme mero mero et oui mi x4 douce aygalades .12 douce aygalades amor corazon douce oh corazon oh douce oh douce oh corazon , oh ! douce oh amor corazon ! oh amor corazon corazon ! corazon , oh douce amor oh douce corazon douce oh ! ! ! , ! douce la oh douce oh douce


EPOCH 1
predict epoch :

[ couplet 1 ] et je zone , comme un loup j'ai trop faim , ouais

predict epoch :

[ couplet 1 ] et je zone , comme un loup j'ai trop faim , ouais de sous et je serre , comme un fou j'suis déter ' , levez-vous j'ai les cernes , je vois flou pour mes frères , j'me mets saoul franchement , je n'sais plus quoi faire trop d'jaloux j'peux plus les compter faire le mac , à quoi ça sert ? tu peux t'faire fumer ou planter j'charme les miss comme calimero en deux-deux j'prends cent numéros ça sert à rien de faire le héros toujours à la base comme mero mero et oui mi corazon amor nez amor nez amor nez j'commence corazon amor corazon djandjaé corazon djandjaé corazon djandjaé corazon djandjaé djandjaé liga nez djandjaé nez amor nez djandjaé amor amor puenta j'ai nez j'ai panique j'commence panique djandjaé corazon j'ai nez amor panique amor panique j'ai amor , nez j'sors corazon liga panique


EPOCH 11
predict epoch :

[ couplet 1 ] et je zone , comme un loup j'ai trop faim , ouais de sous et je serre , comme un fou j'suis déter ' , levez-vous j'ai les cernes ,

predict epoch :

[ couplet 1 ] et je zone , comme un loup j'ai trop faim , ouais de sous et je serre , comme un fou j'suis déter ' , levez-vous j'ai les cernes , je vois flou pour mes frères , j'me mets saoul franchement , je n'sais plus quoi faire trop d'jaloux j'peux plus les compter faire le mac , à quoi ça sert ? tu peux t'faire fumer ou planter j'charme les miss comme calimero en deux-deux j'prends cent numéros ça sert à rien de faire le héros toujours à la base comme mero mero et oui mi nez je sacem je corazon amor sacem amor corazon amor lune shlag couette amor djandjaé , amor j'ai amor shlag djandjaé j'ai amor amor amor j'ai couette shlag djandjaé amor lune j'ai sacem , sacem amor sacem amor djandjaé djandjaé djandjaé , amor j'ai sacem djandjaé amor amor djandjaé amor djandjaé


EPOCH 21
predict epoch :

[ couplet 1 ] et je zone , comme un loup j'ai trop faim , ouais de sous et je serre , comme un fou j'suis déter ' , levez-vous j'ai les cernes , je vois flou pour mes frères , 

In [18]:
def predict(model, words, vocab_to_int, int_to_vocab,nbwords, top_k=5):
    model.eval()
    h = model.init_hidden(1)
    
    for w in words:
        idx = th.LongTensor([[vocab_to_int[w]]])
        out, h = model(idx, h)
    
    _, top_idx = th.topk(out[0], k=top_k)
    choices = top_idx.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])
    
    for _ in range(nbwords):
        ix = th.LongTensor([[choice]])
        out, h = model(idx, h)

        _, top_idx = th.topk(out[0], k=top_k)
        choices = top_idx.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    print(' '.join(words))

In [17]:
predict(model, token[:100], word2idx, idx2word, 30)

[ couplet 1 ] et je zone , comme un loup j'ai trop faim , ouais de sous et je serre , comme un fou j'suis déter ' , levez-vous j'ai les cernes , je vois flou pour mes frères , j'me mets saoul franchement , je n'sais plus quoi faire trop d'jaloux j'peux plus les compter faire le mac , à quoi ça sert ? tu peux t'faire fumer ou planter j'charme les miss comme calimero en deux-deux j'prends cent numéros ça sert à rien de faire le héros toujours à la base comme mero mero et oui mi soir j'essaie m'en , c'est m'en d'garder ken d'garder c'est ken m'en ken j'essaie ken c'est j'essaie m'en c'est m'en m'en ken , m'en m'en j'essaie ] ken c'est j'essaie j'essaie
